In [25]:
!pip install hyperas
!pip install hyperopt

In [26]:
num_classes = 10

In [27]:
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.datasets import mnist
from keras.utils import np_utils

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, Input
from hyperopt import fmin, tpe, hp, anneal, Trials, space_eval, STATUS_OK

In [28]:
def data():# Load the data
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    img_rows, img_cols = 28, 28
    num_classes = 10
    channel = 1
    #(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
    #img_rows, img_cols = 32, 32
    #num_classes = 10
    #channel = 3

    # Add the channel dimension to the images
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], channel, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], channel, img_rows, img_cols)
        input_shape = (channel, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, channel)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, channel)
        input_shape = (img_rows, img_cols, channel)

    print(input_shape)

    # Normalize the pixel values to the range of [0, 1]
    X_train = x_train.astype('float32') / 255
    X_test = x_test.astype('float32') / 255

    # convert class vectors to binary class matrices
    Y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    Y_test = tf.keras.utils.to_categorical(y_test, num_classes)

    return X_train, Y_train, X_test, Y_test

In [29]:
import random
import numpy as np

In [30]:
def model(X_train, Y_train, X_test, Y_test):
    #hp_layers = random.choice([1,2])
    #hp_dropout = random.choice([0, 0.1, 0.2, 0.3, 0.4, 0.5])
    #hp_lr = random.choice(np.random.uniform(0.0001,0.005, 100000))
    #hp_filters1 = np.random.choice([*range(32,128+1, 32)])

    hp_layers = {{choice([1,2])}}
    hp_lr = {{choice(np.arange(0.0001,0.005, 0.0001))}}
    hp_filters1 = {{choice([*range(32,128+1, 32)])}}
    
    model = Sequential()

    # Input layer
    model.add(Conv2D(filters=hp_filters1, kernel_size=3, activation='relu', input_shape=(28,28,1)))
    model.add(BatchNormalization())

    for i in range(hp_layers):
        hp_filters2 = {{choice([*range(32,128+1, 32)])}}
        model.add(Conv2D(filters=hp_filters2, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout({{choice([0, 0.1, 0.2, 0.3, 0.4, 0.5])}}))
    
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}},
                  metrics=['accuracy'])

    model.fit(X_train, Y_train,
              batch_size={{choice([64, 128])}},
              epochs=1,
              verbose=2,
              validation_data=(X_test, Y_test))
    score, acc = model.evaluate(X_test, Y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [31]:
# See: https://stackoverflow.com/questions/49920031/get-the-path-of-the-notebook-on-google-colab
# Install the PyDrive wrapper & import libraries.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Copy/download the file
fid = drive.ListFile({'q':"title='AML -- CNN (TPE).ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('AML -- CNN (TPE).ipynb')

In [32]:
best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          max_evals=1,
                                          algo=tpe.suggest,
                                          notebook_name='AML -- CNN (TPE)', # This is important!
                                          trials=Trials())

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras.models import Sequential, Model
except:
    pass

try:
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, Input
except:
    pass

try:
    from hyperopt import fmin, tpe, hp, a

In [33]:
x_train, y_train, x_test, y_test = data()
history = best_model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test), verbose=1)

(28, 28, 1)
Epoch 1/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.1972 - accuracy: 0.9588 - val_loss: 0.1060 - val_accuracy: 0.9781
Epoch 2/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.1404 - accuracy: 0.9731 - val_loss: 0.0882 - val_accuracy: 0.9827
Epoch 3/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.1127 - accuracy: 0.9781 - val_loss: 0.1321 - val_accuracy: 0.9763
Epoch 4/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0882 - accuracy: 0.9818 - val_loss: 0.0814 - val_accuracy: 0.9844
Epoch 5/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0843 - accuracy: 0.9831 - val_loss: 0.0856 - val_accuracy: 0.9827
Epoch 6/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0644 - accuracy: 0.9865 - val_loss: 0.0638 - val_accuracy: 0.9871
Epoch 7/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0634 - accuracy: 0.9859 - val_lo

In [34]:
TPE_accuracy = history.history['val_accuracy']
TPE_loss = history.history['val_loss']
TPE_training_accuracy = history.history['accuracy']
TPE_training_loss = history.history['loss']

np.save('TPE_MNIST.npy', TPE_accuracy) # save, change based on what dataset is used